In [ ]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
import os
import math
import random
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomRotation(20),transforms.Resize(size=(224,224)),transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
image_datasets = datasets.ImageFolder(root= "/content/drive/My Drive/Dataset unzipped/Stage_2",transform=transform)
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=32, shuffle=True, num_workers=2)



In [ ]:
batch_size=32
test_size=0.3
val_size=0.2



num_data = len(image_datasets)
indices_data = list(range(num_data))
np.random.shuffle(indices_data)
split_tt = int(np.floor(test_size * num_data))
train_idx, test_idx = indices_data[split_tt:], indices_data[:split_tt]

#For Valid
num_train = len(train_idx)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(val_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]


# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_new_idx)
test_sampler = SubsetRandomSampler(test_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size,sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size,sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(image_datasets, sampler = test_sampler, batch_size=batch_size, num_workers=1)
classes = [0,1]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

train_on_gpu = torch.cuda.is_available()

class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()

    self.conv1 = nn.Conv2d(3,16,5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(16,32, 5)
    self.dropout=nn.Dropout(0.2)
    self.fc1 = nn.Linear(32 * 53 * 53, 256)
    self.fc2 = nn.Linear(256, 84)
    self.fc3 = nn.Linear(84, 2)
    self.softmax=nn.LogSoftmax(dim=1)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.dropout(x)
    x = x.view(-1, 32 * 53 * 53)
    x = F.relu(self.fc1(x))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.softmax(self.fc3(x))
    return x

model = Network()
print(model)
if train_on_gpu:
  model.cuda()

Network(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=89888, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [ ]:
import torch.optim as optim
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001,betas=(0.9,0.999),eps=1e-08,weight_decay=0)

In [ ]:
num_epochs=8
train_loss=0.0
for epoch in range(num_epochs):
  model.train()
  for data,label in train_loader:
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
    optimizer.zero_grad()
    output=model(data)
    loss=loss_fn(output,label)
    loss.backward()
    optimizer.step()
    train_loss+=loss.item()*data.size(0)

  train_loss = train_loss/len(train_loader.dataset)
  print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, train_loss))

  

Epoch: 0 	Training Loss: 0.155792
Epoch: 1 	Training Loss: 0.046052
Epoch: 2 	Training Loss: 0.026984
Epoch: 3 	Training Loss: 0.022623
Epoch: 4 	Training Loss: 0.014238
Epoch: 5 	Training Loss: 0.009253
Epoch: 6 	Training Loss: 0.010365
Epoch: 7 	Training Loss: 0.016978


In [ ]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images,labels in valid_loader:
    output=model(images)
    _,predicted=torch.max(output.data,1)
    total+=label.size(0)
    correct += (predicted == labels).sum().item()
  print('Validation Accuracy: {} %'.format((correct / total) * 100))
  print(predicted)
  print(labels)


Validation Accuracy: 100.0 %
tensor([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0])
tensor([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0])


In [ ]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_loader:
    output=model(images)
    _,predicted=torch.max(output.data,1)
    total+=label.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy: {} %'.format((correct / total) * 100))
  print(predicted)
  print(labels)


Test Accuracy: 89.58333333333334 %
tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 0, 0, 0])
tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 0])
